In [20]:

import gensim
import re
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import matplotlib.pyplot as plt
# from wordcloud import WordCloud
from nltk import word_tokenize
import string
string.punctuation

from gensim.parsing.preprocessing import remove_stopwords
# import nltk
# nltk.download('punkt')
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np

from sklearn.metrics import*

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('yelp_labelled.txt',sep=".\t",header=None)

<ipython-input-2-8d40498fd0f1>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('yelp_labelled.txt',sep=".\t",header=None)


In [3]:
def preprocess(text):
    text = re.sub(re.compile('<.*?>') , '', text)
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = text.lower()
    text = "".join([t for t in text if t not in string.punctuation])
    text = remove_stopwords(text)
    # text = word_tokenize(text)
    return np.array(text)

def avgw2v(text,dim,model):
    n = len(text)
    sumVector = np.zeros(dim)

    for t in text:
        sumVector = np.add(sumVector,model.wv[t])
    return np.divide(sumVector,n)

In [4]:
df[0]=df[0].apply(lambda x:preprocess(x))

In [5]:
tokenizedReviews = df[0].apply(gensim.utils.simple_preprocess)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(np.array(tokenizedReviews),df[1], test_size=0.2,stratify=df[1],random_state=42)


In [8]:
words = []
for t in x_train:
    for w in t:
        if w not in words:
            words.append(w)

In [9]:
words.sort()

In [10]:
len(words)

1569

In [11]:
TrainMat = np.zeros((len(x_train),len(words)))

In [12]:
TrainMat[1][576]

0.0

In [13]:
x_train[0]

['perfect', 'sitdown', 'family', 'meal', 'friends']

In [14]:
x_train[0]

['perfect', 'sitdown', 'family', 'meal', 'friends']

In [15]:

for i in range(len(x_train)):
    
    for w in x_train[i]:
        # print(w)
        idx = words.index(w)
        # print(idx)
        TrainMat[i][idx]+=1



In [16]:
TestMat = np.zeros((len(x_test),len(words)))
for i in range(len(x_test)):
    
    for w in x_test[i]:
        if w in words:
            idx = words.index(w)
            TestMat[i][idx]+=1

In [23]:
clf = MultinomialNB(alpha=1)
clf.fit(TrainMat,y_train)


MultinomialNB(alpha=1)

In [18]:
clf.score(TestMat,y_test)

0.82

In [19]:
clf.score(TrainMat,y_train)

0.95625

In [21]:
f1_score(y_test, clf.predict(TestMat))

0.826923076923077

In [22]:
f1_score(y_train, clf.predict(TrainMat))

0.9561952440550688